<a href="https://colab.research.google.com/github/fidanaydamirova/youtube_webscraping/blob/main/Youtube_comments_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Introduction: We are performing webscraping of youtube video comments, storing them in a dataset and performing sentiment analysis on them to find the percentage of positive and negative reviews.***

---






In [ ]:
!pip install time
!pip install nltk
!pip install selenium

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [ ]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (95.0.4638.69-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [ ]:
#import all required libraries
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
from statistics import mean

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
# set options to be headless, ..
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


In [ ]:
data = []
# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)
driver.get("https://www.youtube.com/watch?v=CtXEcBz3Go0&t=1s")
# scroll till the end of the video so comments will start loading
driver.execute_script('window.scrollTo(1, 500);')
# now wait let load the comments
time.sleep(5)
with driver:
    wait = WebDriverWait(driver, 5)

    #we can increase the range to get more comments
    for item in range(40):
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(5)

    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

df = pd.DataFrame(data, columns=['comment'])

In [ ]:
#take a look at the top of the data set
df.head(10)

,comment
0,TW\nSKIP NAVIGATION\nSIGN IN\n0:01 / 1:42\n#Ex...
1,
2,"Check out the official Extremely Wicked, Shock..."
3,本片從泰德交往最久的女友伊莉莎白的角度切入，即使罪證確鑿，伊莉莎白多年來仍拒絕接受有關泰德邦...
4,Life wasn’t so easy for Troy Bolton after grad...
5,Oh god when the little girl went “ted!” I was ...
6,For everyone that's upset about the romanticiz...
7,Everyone who is saying this movie romanticized...
8,I'm so skeptical right now. I'm glad Zac Efron...
9,I don't get the hate...it's seems like a good ...


In [ ]:
#drop top rows that don't contain comments
df = df.drop(df.index[range(4)])
df.head()

,comment
4,Life wasn’t so easy for Troy Bolton after grad...
5,Oh god when the little girl went “ted!” I was ...
6,For everyone that's upset about the romanticiz...
7,Everyone who is saying this movie romanticized...
8,I'm so skeptical right now. I'm glad Zac Efron...


In [ ]:
#check how many comments we have
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 801 entries, 4 to 804
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  801 non-null    object
dtypes: object(1)
memory usage: 12.5+ KB


In [ ]:
nltk.downloader.download('vader_lexicon')
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
#define new column "scores" that contains the info of positive, 
#negative and neutral proportion of a comment lines
df['scores'] = df['comment'].apply(lambda comment: vader.polarity_scores(comment))

#look at the top of the data
df.head()

,comment,scores
4,Life wasn’t so easy for Troy Bolton after grad...,"{'neg': 0.0, 'neu': 0.681, 'pos': 0.319, 'comp..."
5,Oh god when the little girl went “ted!” I was ...,"{'neg': 0.189, 'neu': 0.541, 'pos': 0.27, 'com..."
6,For everyone that's upset about the romanticiz...,"{'neg': 0.16, 'neu': 0.662, 'pos': 0.178, 'com..."
7,Everyone who is saying this movie romanticized...,"{'neg': 0.049, 'neu': 0.818, 'pos': 0.133, 'co..."
8,I'm so skeptical right now. I'm glad Zac Efron...,"{'neg': 0.05, 'neu': 0.782, 'pos': 0.168, 'com..."


In [ ]:
# add new column "compound" which is the sum of positive, negative & neutral 
# scores which is then normalized between -1(most extreme negative) 
# and +1 (most extreme positive)
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])

df.head()

,comment,scores,compound
4,Life wasn’t so easy for Troy Bolton after grad...,"{'neg': 0.0, 'neu': 0.681, 'pos': 0.319, 'comp...",0.5777
5,Oh god when the little girl went “ted!” I was ...,"{'neg': 0.189, 'neu': 0.541, 'pos': 0.27, 'com...",0.4406
6,For everyone that's upset about the romanticiz...,"{'neg': 0.16, 'neu': 0.662, 'pos': 0.178, 'com...",0.2332
7,Everyone who is saying this movie romanticized...,"{'neg': 0.049, 'neu': 0.818, 'pos': 0.133, 'co...",0.4767
8,I'm so skeptical right now. I'm glad Zac Efron...,"{'neg': 0.05, 'neu': 0.782, 'pos': 0.168, 'com...",0.7394


In [ ]:
# check if the overall comment negative or positive
df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

df.head()

,comment,scores,compound,comp_score
4,Life wasn’t so easy for Troy Bolton after grad...,"{'neg': 0.0, 'neu': 0.681, 'pos': 0.319, 'comp...",0.5777,pos
5,Oh god when the little girl went “ted!” I was ...,"{'neg': 0.189, 'neu': 0.541, 'pos': 0.27, 'com...",0.4406,pos
6,For everyone that's upset about the romanticiz...,"{'neg': 0.16, 'neu': 0.662, 'pos': 0.178, 'com...",0.2332,pos
7,Everyone who is saying this movie romanticized...,"{'neg': 0.049, 'neu': 0.818, 'pos': 0.133, 'co...",0.4767,pos
8,I'm so skeptical right now. I'm glad Zac Efron...,"{'neg': 0.05, 'neu': 0.782, 'pos': 0.168, 'com...",0.7394,pos


In [ ]:
# here we can explore some examples from our data set and compare the results 
# which scores are consireder positive and which negative
for i in range(6,20):
    print(vader.polarity_scores(df["comment"][i]))

{'neg': 0.16, 'neu': 0.662, 'pos': 0.178, 'compound': 0.2332}
{'neg': 0.049, 'neu': 0.818, 'pos': 0.133, 'compound': 0.4767}
{'neg': 0.05, 'neu': 0.782, 'pos': 0.168, 'compound': 0.7394}
{'neg': 0.0, 'neu': 0.76, 'pos': 0.24, 'compound': 0.802}
{'neg': 0.078, 'neu': 0.819, 'pos': 0.102, 'compound': 0.579}
{'neg': 0.307, 'neu': 0.526, 'pos': 0.167, 'compound': -0.3818}
{'neg': 0.085, 'neu': 0.712, 'pos': 0.203, 'compound': 0.9062}
{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.6901}
{'neg': 0.097, 'neu': 0.702, 'pos': 0.201, 'compound': 0.9046}
{'neg': 0.259, 'neu': 0.569, 'pos': 0.173, 'compound': -0.6597}
{'neg': 0.0, 'neu': 0.645, 'pos': 0.355, 'compound': 0.8874}
{'neg': 0.081, 'neu': 0.699, 'pos': 0.22, 'compound': 0.8212}
{'neg': 0.0, 'neu': 0.475, 'pos': 0.525, 'compound': 0.9245}
{'neg': 0.155, 'neu': 0.726, 'pos': 0.119, 'compound': -0.754}


In [ ]:
# get the mean of positive comments
mean(df["comp_score"] == "pos")

0.7553058676654182

In [ ]:
# get the mean of negative comments
mean(df["comp_score"] == "neg")

0.24469413233458176

In [ ]:
import matplotlib.pyplot as plt

# Conclusion: Overall we can conclude that this particular video has 80.7% positive and 19.3% negative comments.